## Imports

In [1]:
import sys
import json
import pandas as pd
sys.path.append('../')
from lib.evaluation import TextReuseEvaluator
from pathlib import Path

## Configuration

In [61]:
pd.set_option('display.max_colwidth', None)

In [81]:
PASSIM_EXPERIMENTS_CONFIG_PATH = '../data/experiments_passim.json'
SERIATIM_EXPERIMENTS_CONFIG_PATH = '../data/experiments_seriatim.json'
GT_DATASET_PATH = '../data/homeric_repetitions_dataset.tsv'

In [86]:
config = json.loads(Path(PASSIM_EXPERIMENTS_CONFIG_PATH).read_text())

In [87]:
config += json.loads(Path(SERIATIM_EXPERIMENTS_CONFIG_PATH).read_text())

## Print commands

In [88]:
for experiment in config:
    command = " ".join([
        experiment['algorithm'],
        experiment['parameters'],
        experiment['input_path'],
        experiment['output_path']
    ])
    print(experiment['label'])
    print(command)

exp0
passim -n 1 --min-match 1 -a 5 /Users/mromanel/Documents/repetitions-homeric-speeches/data/input/homeric_speeches_raw.json /Users/mromanel/Documents/repetitions-homeric-speeches/data/passim/exp0/
exp4
passim -n 2 --min-match 2 -a 10 /Users/mromanel/Documents/repetitions-homeric-speeches/data/input/homeric_speeches_raw.json /Users/mromanel/Documents/repetitions-homeric-speeches/data/passim/exp4/
exp5
passim -n 3 --min-match 1 -a 10 /Users/mromanel/Documents/repetitions-homeric-speeches/data/input/homeric_speeches_raw.json /Users/mromanel/Documents/repetitions-homeric-speeches/data/passim/exp5/
exp6
passim -n 4 --min-match 1 -a 10 /Users/mromanel/Documents/repetitions-homeric-speeches/data/input/homeric_speeches_raw.json /Users/mromanel/Documents/repetitions-homeric-speeches/data/passim/exp6-redo/
exp7
passim -n 3 --min-match 2 -a 10 /Users/mromanel/Documents/repetitions-homeric-speeches/data/input/homeric_speeches_raw.json /Users/mromanel/Documents/repetitions-homeric-speeches/data

## Run evaluation

In [66]:
#config

In [89]:
data = []

for experiment in config:
    print(f"\n#######\n{experiment['label']}\n#######\n")
    print(experiment['algorithm'])
    print(experiment['parameters'])
    print('\n')
    evaluator = TextReuseEvaluator()
    evaluator.read_predictions(experiment['json_output_path'])
    evaluator.read_groundtruth(GT_DATASET_PATH)
    results = evaluator.evaluate()
    #evaluator.print_summary()
    results['label'] = experiment['label']
    results['algorithm'] = experiment['algorithm']
    data.append(results)


#######
exp0
#######

passim
-n 1 --min-match 1 -a 5


7993 predicted clusters found
68 groundtruth clusters found

#######
exp4
#######

passim
-n 2 --min-match 2 -a 10


2078 predicted clusters found
68 groundtruth clusters found

#######
exp5
#######

passim
-n 3 --min-match 1 -a 10


431 predicted clusters found
68 groundtruth clusters found

#######
exp6
#######

passim
-n 4 --min-match 1 -a 10


341 predicted clusters found
68 groundtruth clusters found

#######
exp7
#######

passim
-n 3 --min-match 2 -a 10


431 predicted clusters found
68 groundtruth clusters found

#######
exp6-lemmat
#######

passim
-n 4 --min-match 1 -a 10 --max-repeat 100


364 predicted clusters found
68 groundtruth clusters found

#######
exp6-lemmat-filter
#######

passim
-n 4 --min-match 1 -a 10 --max-repeat 100


284 predicted clusters found
68 groundtruth clusters found

#######
exp12
#######

seriatim
-n 5 --src-overlap 0.8 --dst-overlap 0.8 --min-match 1 -a 10 --minDF 2 --maxDF 100


862 predicted 

In [90]:
filter_columns = [
    'algorithm',
    'pct_matched_clusters',
    'pct_exact_clusters',
    'pct_partial_clusters',
    'pct_spurious_clusters',
    'precision',
    'recall',
    'f1-score'
]
eval_df = pd.DataFrame(data).set_index('label')[filter_columns].rename(columns={
    'algorithm': 'Version',
    'label':'Configuration',
    'pct_matched_clusters': 'Matched clust. (%)',
    'pct_exact_clusters': 'Exact clust. (%)',
    'pct_partial_clusters':'Partial clust. (%)',
    'pct_spurious_clusters': 'Spurious clust. (%)',
    'precision':'Precision',
    'recall': 'Recall',
    'f1-score': 'F1-score'
})

In [91]:
eval_df.index.rename('Configuration', inplace=True)

In [92]:
eval_df

,Version,Matched clust. (%),Exact clust. (%),Partial clust. (%),Spurious clust. (%),Precision,Recall,F1-score
Configuration,,,,,,,,
exp0,passim,33.82,4.35,26.09,91.30,0.517,0.313,0.390
exp4,passim,70.59,50.00,6.25,41.67,0.780,0.701,0.738
exp5,passim,89.71,86.89,4.92,4.92,0.978,0.898,0.936
exp6,passim,89.71,90.16,4.92,1.64,0.992,0.898,0.943
exp7,passim,89.71,86.89,4.92,4.92,0.978,0.898,0.936
exp6-lemmat,passim,89.71,88.52,4.92,3.28,0.985,0.898,0.940
exp6-lemmat-filter,passim,82.35,87.50,7.14,1.79,0.992,0.823,0.900
exp12,seriatim,35.29,75.00,4.17,20.83,0.873,0.327,0.475
exp9,seriatim,75.00,82.35,7.84,7.84,0.964,0.735,0.834


In [93]:
pd.DataFrame(data)

,n_predicted_clusters,n_matched_clusters,n_unmatched_clusters,n_exact_clusters,n_partial_clusters,n_spurious_clusters,n_gt_passages,n_matched_passages,n_missed_passages,n_spurious_passages,pct_matched_clusters,pct_exact_clusters,pct_partial_clusters,pct_spurious_clusters,precision,recall,f1-score,label,algorithm
0,7993,23,45,1,6,21,147,46,101,43,33.82,4.35,26.09,91.30,0.517,0.313,0.390,exp0,passim
1,2078,48,20,24,3,20,147,103,44,29,70.59,50.00,6.25,41.67,0.780,0.701,0.738,exp4,passim
2,431,61,7,53,3,3,147,132,15,3,89.71,86.89,4.92,4.92,0.978,0.898,0.936,exp5,passim
3,341,61,7,55,3,1,147,132,15,1,89.71,90.16,4.92,1.64,0.992,0.898,0.943,exp6,passim
4,431,61,7,53,3,3,147,132,15,3,89.71,86.89,4.92,4.92,0.978,0.898,0.936,exp7,passim
5,364,61,7,54,3,2,147,132,15,2,89.71,88.52,4.92,3.28,0.985,0.898,0.940,exp6-lemmat,passim
6,284,56,12,49,4,1,147,121,26,1,82.35,87.50,7.14,1.79,0.992,0.823,0.900,exp6-lemmat-filter,passim
7,862,24,44,18,1,5,147,48,99,7,35.29,75.00,4.17,20.83,0.873,0.327,0.475,exp12,seriatim
8,305,51,17,42,4,4,147,108,39,4,75.00,82.35,7.84,7.84,0.964,0.735,0.834,exp9,seriatim
9,314,51,17,43,2,6,147,108,39,7,75.00,84.31,3.92,11.76,0.939,0.735,0.824,exp9a,seriatim


In [94]:
pd.DataFrame(data).set_index('label').rename(columns={
    'algorithm': 'Version',
    'label':'Configuration',
    'pct_matched_clusters': 'Matched clust. (%)',
    'pct_exact_clusters': 'Exact clust. (%)',
    'pct_partial_clusters':'Partial clust. (%)',
    'pct_spurious_clusters': 'Spurious clust. (%)',
    'precision':'Precision',
    'recall': 'Recall',
    'f1-score': 'F1-score'
})

,n_predicted_clusters,n_matched_clusters,n_unmatched_clusters,n_exact_clusters,n_partial_clusters,n_spurious_clusters,n_gt_passages,n_matched_passages,n_missed_passages,n_spurious_passages,Matched clust. (%),Exact clust. (%),Partial clust. (%),Spurious clust. (%),Precision,Recall,F1-score,Version
label,,,,,,,,,,,,,,,,,,
exp0,7993,23,45,1,6,21,147,46,101,43,33.82,4.35,26.09,91.30,0.517,0.313,0.390,passim
exp4,2078,48,20,24,3,20,147,103,44,29,70.59,50.00,6.25,41.67,0.780,0.701,0.738,passim
exp5,431,61,7,53,3,3,147,132,15,3,89.71,86.89,4.92,4.92,0.978,0.898,0.936,passim
exp6,341,61,7,55,3,1,147,132,15,1,89.71,90.16,4.92,1.64,0.992,0.898,0.943,passim
exp7,431,61,7,53,3,3,147,132,15,3,89.71,86.89,4.92,4.92,0.978,0.898,0.936,passim
exp6-lemmat,364,61,7,54,3,2,147,132,15,2,89.71,88.52,4.92,3.28,0.985,0.898,0.940,passim
exp6-lemmat-filter,284,56,12,49,4,1,147,121,26,1,82.35,87.50,7.14,1.79,0.992,0.823,0.900,passim
exp12,862,24,44,18,1,5,147,48,99,7,35.29,75.00,4.17,20.83,0.873,0.327,0.475,seriatim
exp9,305,51,17,42,4,4,147,108,39,4,75.00,82.35,7.84,7.84,0.964,0.735,0.834,seriatim


In [95]:
filter_columns = [
    'algorithm',
    'n_predicted_clusters',
    'n_gt_passages',
    'n_matched_passages',
    'n_missed_passages',
    'n_spurious_passages',
    'precision',
    'recall',
    'f1-score'
]
pd.DataFrame(data).set_index('label')[filter_columns].rename(columns={
    'algorithm': 'Version',
    'label':'Configuration',
    'precision':'Precision',
    'recall': 'Recall',
    'f1-score': 'F1-score'
})

,Version,n_predicted_clusters,n_gt_passages,n_matched_passages,n_missed_passages,n_spurious_passages,Precision,Recall,F1-score
label,,,,,,,,,
exp0,passim,7993,147,46,101,43,0.517,0.313,0.390
exp4,passim,2078,147,103,44,29,0.780,0.701,0.738
exp5,passim,431,147,132,15,3,0.978,0.898,0.936
exp6,passim,341,147,132,15,1,0.992,0.898,0.943
exp7,passim,431,147,132,15,3,0.978,0.898,0.936
exp6-lemmat,passim,364,147,132,15,2,0.985,0.898,0.940
exp6-lemmat-filter,passim,284,147,121,26,1,0.992,0.823,0.900
exp12,seriatim,862,147,48,99,7,0.873,0.327,0.475
exp9,seriatim,305,147,108,39,4,0.964,0.735,0.834


## Experiment table

In [29]:
configurations_df = pd.DataFrame(config).set_index('label')[['algorithm', 'parameters']].rename(columns={
    'algorithm':'Version',
    'parameters':'Parameters'
})

In [32]:
configurations_df.index.rename('Configuration', inplace=True)

In [33]:
configurations_df

,Version,Parameters
Configuration,,
conf-0,passim,-n 1 --min-match 1 -a 5
conf-4,passim,-n 2 --min-match 2 --max-repeat 100 -a 10
conf-5,passim,-n 3 --min-match 1 --max-repeat 100 -a 10
conf-6,passim,-n 4 --min-match 1 --max-repeat 100 -a 10
conf-7,passim,-n 3 --min-match 2 --max-repeat 100 -a 10
conf-8,seriatim,-n 10 --min-match 1 -a 10 --minDF 2 --maxDF 100
conf-8a,seriatim,-n 10 --min-match 1 -a 5 --minDF 2 --maxDF 100
conf-8b,seriatim,-n 10 --min-match 1 -a 50 --minDF 2 --maxDF 100
conf-8c,seriatim,-n 10 --min-match 1 -a 60 --minDF 2 --maxDF 100
